In [ ]:
import pandas as pd         # 导入pandas库，它就像一个超级强大的Excel工具，用来处理表格数据非常方便。
import numpy as np          # 导入numpy库，它主要用于进行数值计算，特别是在处理数组（例如一堆数字）时效率很高。
import matplotlib.pyplot as plt # 导入matplotlib.pyplot库，这个库是用来画图的，比如折线图、柱状图等，让数据可视化。

# 读取数据集 2分
data = pd.read_csv('sensor_data.csv') # 使用pandas的read_csv函数，读取名为 'sensor_data.csv' 的文件。
                                       # 这就相当于我们打开这个CSV文件，把里面的内容读取到一个叫做 'data' 的表格（DataFrame）里。
print(data)                            # 打印出 'data' 这个表格的内容，让我们能看到原始数据的样子。

# 1. 传感器数据统计
# 对传感器类型进行分组，并计算每个组的数据数量和平均值 3分
sensor_stats = data.groupby(data['SensorType'])['Value'].agg(['count','mean'])
# 让我们一步步拆解这行代码：
# data.groupby(data['SensorType'])：这行代码的意思是，把我们的大表格 'data' 按照 'SensorType'（传感器类型）这一列的值进行分组。
# 比如，如果数据里有 'Temperature'（温度）和 'Humidity'（湿度）两种传感器，它就会把所有温度数据分到一组，所有湿度数据分到另一组。
# ['Value']：在每个分组内部，我们只关心 'Value'（传感器数值）这一列的数据。
# .agg(['count','mean'])：'agg' 是 'aggregate' 的缩写，意思是“聚合”。
# 它会把之前分组后的 'Value' 列数据做两个计算：
#   'count'：统计每个分组里有多少个数据点，也就是某个类型传感器有多少条记录。
#   'mean'：计算每个分组里 'Value' 的平均值，也就是某个类型传感器的平均数值。
# 最终，'sensor_stats' 这个变量就会存储一个统计表，显示每种传感器类型的数据数量和平均值。

# 输出结果
print("传感器数据数量和平均值:") # 打印一个标题，说明接下来要显示什么。
print(sensor_stats)                # 打印出 'sensor_stats' 这个统计表的内容。

# 2. 按位置统计温度和湿度数据
# 筛选出温度和湿度数据，然后按位置和传感器类型分组，计算每个组的平均值 2分
location_stats = data[data['SensorType'].isin(['Temperature','Humidity'])].groupby(['Location','SensorType'])['Value'].mean().unstack()
# 这行代码稍微复杂一些，我们慢慢来分解：
# data['SensorType'].isin(['Temperature','Humidity'])：这是一个筛选条件。
# 它检查 'SensorType' 这一列的值，看它是不是在列表 `['Temperature','Humidity']` 里面。
# 如果是，结果就是 True；如果不是，结果就是 False。
# data[...]：用上面生成的 True/False 系列来筛选原始的 'data' 表格。
# 只有那些 'SensorType' 是 'Temperature' 或 'Humidity' 的行才会被保留下来。
# .groupby(['Location','SensorType'])：对筛选后的数据进行分组，这次是同时按照 'Location'（位置）和 'SensorType'（传感器类型）两列进行分组。
# 比如，会有一个“客厅-温度”组，一个“客厅-湿度”组，一个“卧室-温度”组等等。
# ['Value'].mean()：在每个分组内部，计算 'Value' 列的平均值。
# .unstack()：这是一个非常酷的pandas操作，它的作用是“旋转”表格。
# 想象一下，分组后的结果可能是：
# Location  SensorType
# 客厅      Humidity   25.0
#         Temperature 22.5
# 卧室      Humidity   30.0
#         Temperature 20.0
# unstack() 会把 'SensorType' 这一层级从行索引变成列标题，让表格看起来更像我们常用的交叉表：
# SensorType  Humidity  Temperature
# Location
# 客厅           25.0        22.5
# 卧室           30.0        20.0
# 这样，我们就能更直观地看到每个位置的平均温度和平均湿度。
# 最终，'location_stats' 存储的就是这个经过旋转的平均值表格。

# 输出结果
print("每个位置的温度和湿度数据平均值:") # 打印一个标题。
print(location_stats)                  # 打印出 'location_stats' 这个统计表的内容。

# 3. 数据清洗和异常值处理
# 标记异常值 3分
data['is_abnormal'] = np.where(
    # 这是一个条件非常复杂的 np.where，用来判断每个数据是不是异常值。
    # 我们一层层地看：
    # 外层 np.where(条件, 如果条件为真赋的值, 如果条件为假赋的值)
    # 如果条件成立，data['is_abnormal'] 列会赋值为 True，否则赋值为 False。

    # 来看“条件”的部分：
    ((data['SensorType']=='Temperature') & ((data['Value'] < -10) | (data['Value'] > 50))) |
    # 这一部分是针对温度传感器数据的异常值判断：
    # (data['SensorType']=='Temperature')：首先判断传感器类型是不是“Temperature”（温度）。
    # & ( ... )：如果确实是温度传感器，接着判断后面的条件（逻辑“与”）。
    # ((data['Value'] < -10) | (data['Value'] > 50))：判断温度值是不是小于-10度 或 大于50度（逻辑“或”）。
    # 也就是说，如果传感器是温度类型，并且数值低于-10或高于50，那么它就是异常。

    ((data['SensorType']=='Humidity') & ((data['Value'] < 0) | (data['Value'] > 100))),
    # 这一部分是针对湿度传感器数据的异常值判断：
    # (data['SensorType']=='Humidity')：判断传感器类型是不是“Humidity”（湿度）。
    # & ( ... )：如果确实是湿度传感器，接着判断后面的条件。
    # ((data['Value'] < 0) | (data['Value'] > 100))：判断湿度值是不是小于0% 或 大于100%（因为湿度百分比通常在0到100之间）。
    # 也就是说，如果传感器是湿度类型，并且数值低于0或高于100，那么它就是异常。

    # 最后的逻辑是：
    # 第一个椭圆(...) | 第二个椭圆(...)：两个大条件的逻辑“或”。
    # 意味着，只要满足“温度异常”或“湿度异常”中的任何一个，整个条件就为 True。
    True, # 如果上面的整个大条件为 True，那么'is_abnormal'列对应的值就是 True。
    False # 如果上面的整个大条件为 False，那么'is_abnormal'列对应的值就是 False。
)
# 总结：这行代码就是根据传感器的类型和数值范围，智能化地给每条数据贴上“是不是异常值”的标签。

# 输出异常值数量 2分
print("异常值数量:", data['is_abnormal'].sum())
# data['is_abnormal']：包含 True/False 标签的列（True表示异常，False表示正常）。
# .sum()：对这个布尔值（True/False）系列进行求和。在Python中，True被看作1，False被看作0。
# 所以，求和的结果就是所有 True 的数量，也就是异常值的总数量。
# 这种方法比 .value_counts() 更直接地获取异常值的总数。

# 填补缺失值
# 使用前向填充和后向填充的方法填补缺失值 4分
data['Value'].fillna(method='ffill', inplace=True)
# data['Value']：我们关注的是 'Value'（传感器数值）这一列，因为数据通常会在这一列出现缺失。
# .fillna(method='ffill', inplace=True)：这是用来填充缺失值（NaN）的方法。
# method='ffill'：'ffill' 是 'forward fill' 的缩写，意思是“前向填充”。
# 它会用前一个有效的数据值来填充当前遇到的缺失值。
# 举例：如果数据是 [10, NaN, 12]，经过 'ffill' 填充后会变成 [10, 10, 12]。
# inplace=True：这个参数很重要，它表示直接修改原始的 'data' DataFrame，而不是创建一个新的修改后的副本。
# 也就是说，这行代码会直接修改 'data' 表格的 'Value' 列。

data['Value'].fillna(method='bfill', inplace=True)
# .fillna(method='bfill', inplace=True)：这和上面一行类似，但是使用了 'bfill'。
# method='bfill'：'bfill' 是 'backward fill' 的缩写，意思是“后向填充”。
# 它会用后一个有效的数据值来填充当前遇到的缺失值。
# 举例：如果数据是 [NaN, 10, 12]，经过 'bfill' 填充后会变成 [10, 10, 12]。
# 为什么要用两次填充呢？因为 'ffill' 无法填充第一个值就是NaN的情况，'bfill' 无法填充最后一个值是NaN的情况。
# 组合使用可以更全面地处理缺失值，确保所有缺失值都被尽可能合理地填补。

# 保存清洗后的数据
# 删除用于标记异常值的列，并将清洗后的数据保存到新的CSV文件中 4分
cleaned_data = data.drop(columns=['is_abnormal'])
# data.drop(columns=['is_abnormal'])：'drop' 方法用于删除DataFrame中的行或列。
# columns=['is_abnormal']：指定我们要删除的是名为 'is_abnormal' 的列。
# 因为 'is_abnormal' 列只是我们在数据清洗过程中用于标记异常值的临时列，现在数据处理完了，就不需要它了。
# cleaned_data = ...：将删除列后的新DataFrame赋值给 'cleaned_data' 这个变量。

cleaned_data.to_csv('cleaned_sensor_data.csv', index=False)
# cleaned_data.to_csv(...)：这是pandas用来将DataFrame保存为CSV文件的方法。
# 'cleaned_sensor_data.csv'：指定新保存的文件的名字。
# index=False：这个参数很重要，它表示在保存CSV文件时，不要把DataFrame的索引（左边默认从0开始的数字序号）也写入到CSV文件中。
# 如果不设置这个参数，CSV文件里会多一列无用的索引号。
print("数据清洗完成，已保存为 'cleaned_sensor_data.csv'") # 打印一条消息，告诉用户数据已经清洗完成并保存。
